In [13]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [15]:
if 'ygong' in os.getcwd():
    filepath = "../data"
    dir_out = "../output"
else:
    filepath = "/home/gong/Documents/Kaggle_July2019/data"
    dir_out = "/home/gong/Documents/Kaggle_July2019/output"

def load_data(filepath):
    train = pd.read_csv(os.path.join(filepath, 'train.csv'))
    test = pd.read_csv(os.path.join(filepath, 'test.csv'))
    submit = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
    structures = pd.read_csv(os.path.join(filepath, 'structures.csv'))

    print('Train dataset shape is -> rows: {} cols:{}'.format(train.shape[0], train.shape[1]))
    print('Test dataset shape is  -> rows: {} cols:{}'.format(test.shape[0], test.shape[1]))
    print('Sample submission dataset shape is  -> rows: {} cols:{}'.format(submit.shape[0], submit.shape[1]))
    print('Structures dataset shape is  -> rows: {} cols:{}'.format(structures.shape[0], structures.shape[1]))
    print('\n')

    return train, test, submit, structures

train, test, submit, structures = load_data(filepath)

Train dataset shape is -> rows: 4658147 cols:6
Test dataset shape is  -> rows: 2505542 cols:5
Sample submission dataset shape is  -> rows: 2505542 cols:2
Structures dataset shape is  -> rows: 2358657 cols:6




In [16]:
def molecule_properties(structures):
    atomic_radius = {'H': 0.38, 'C': 0.77, 'N': 0.75, 'O': 0.73, 'F': 0.71}
    fudge_factor = 0.05
    atomic_radius = {k: v + fudge_factor for k, v in atomic_radius.items()}
    print(atomic_radius)

    electronegativity = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}

    atoms = structures['atom'].values
    atoms_en = [electronegativity[x] for x in tqdm(atoms)] # electronegrativity
    atoms_rad = [atomic_radius[x] for x in tqdm(atoms)]

    structures['EN'] = atoms_en
    structures['rad'] = atoms_rad

    return structures

structures = molecule_properties(structures)

  6%|▌         | 143578/2358657 [00:00<00:01, 1435777.95it/s]

{'H': 0.43, 'C': 0.8200000000000001, 'N': 0.8, 'O': 0.78, 'F': 0.76}


100%|██████████| 2358657/2358657 [00:01<00:00, 1873351.26it/s]


In [17]:
def bond_length(structures):
    i_atom = structures['atom_index'].values
    p = structures[['x', 'y', 'z']].values
    p_compare = p
    m = structures['molecule_name'].values
    m_compare = m
    r = structures['rad'].values
    r_compare = r

    source_row = np.arange(len(structures))
    max_atoms = 28

    bonds = np.zeros((len(structures) + 1, max_atoms + 1), dtype=np.int8)
    bond_dists = np.zeros((len(structures) + 1, max_atoms + 1), dtype=np.float32)

    print('Calculating the bonds')

    for i in tqdm(range(max_atoms - 1)):
        p_compare = np.roll(p_compare, -1, axis=0)
        m_compare = np.roll(m_compare, -1, axis=0)
        r_compare = np.roll(r_compare, -1, axis=0)

        # Are we still comparing atoms in the same molecule?
        mask = np.where(m == m_compare, 1, 0)
        dists = np.linalg.norm(p - p_compare, axis=1) * mask
        r_bond = r + r_compare

        bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)

        source_row = source_row
        # Note: Will be out of bounds of bonds array for some values of i
        target_row = source_row + i + 1
        # If invalid target, write to dummy row
        target_row = np.where(np.logical_or(target_row > len(structures), mask == 0), len(structures),
                              target_row)  # If invalid target, write to dummy row

        source_atom = i_atom
        # Note: Will be out of bounds of bonds array for some values of i
        target_atom = i_atom + i + 1
        # If invalid target, write to dummy col
        target_atom = np.where(np.logical_or(target_atom > max_atoms, mask == 0), max_atoms,
                               target_atom)

        bonds[(source_row, target_atom)] = bond
        bonds[(target_row, source_atom)] = bond
        bond_dists[(source_row, target_atom)] = dists
        bond_dists[(target_row, source_atom)] = dists

    bonds = np.delete(bonds, axis=0, obj=-1)  # Delete dummy row
    bonds = np.delete(bonds, axis=1, obj=-1)  # Delete dummy col
    bond_dists = np.delete(bond_dists, axis=0, obj=-1)  # Delete dummy row
    bond_dists = np.delete(bond_dists, axis=1, obj=-1)  # Delete dummy col

    print('Counting and condensing bonds')

    bonds_numeric = [[i for i, x in enumerate(row) if x] for row in tqdm(bonds)]
    bond_lengths = [[dist for i, dist in enumerate(row) if i in bonds_numeric[j]] for j, row in
                    enumerate(tqdm(bond_dists))]
    bond_lengths_mean = [np.mean(x) for x in bond_lengths]
    n_bonds = [len(x) for x in bonds_numeric]

    bond_data = {'n_bonds': n_bonds, 'bond_lengths_mean': bond_lengths_mean}
    bond_df = pd.DataFrame(bond_data)
    structures = structures.join(bond_df)
    # save data
    # structures.to_csv(os.path.join(self.filepath, 'molecular_structure.csv'))

    return structures

structures = bond_length(structures)

  0%|          | 0/27 [00:00<?, ?it/s]

Calculating the bonds


  1%|          | 22693/2358657 [00:00<00:20, 112568.29it/s]

Counting and condensing bonds


100%|██████████| 2358657/2358657 [00:28<00:00, 82228.23it/s]


In [18]:
# calculate bond length adopted from @Chanran Kim - Kaggle kernel
def map_atom_info(df, atom_idx, structures):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0, structures)
train = map_atom_info(train, 1, structures)
test = map_atom_info(test, 0, structures)
test = map_atom_info(test, 1, structures)

In [19]:
def calculate_dist(df):
    p_0 = df[['x_0', 'y_0', 'z_0']].values
    p_1 = df[['x_1', 'y_1', 'z_1']].values
    df['dist'] = np.linalg.norm(p_0 - p_1, axis=1)
    return df
    
def get_intervene_bonds(df):
    J_type = df['type'].values
    intervene_bonds = [int(item[0]) for item in tqdm(J_type)]
    df['intervene_bonds'] = intervene_bonds    
    return df

print("Calculate the distance between two atoms")
train = calculate_dist(train)
test = calculate_dist(test)

# extract the number of intervening bonds
print('Extracting number of intervene bonds from type')
train = get_intervene_bonds(train)
test = get_intervene_bonds(test)

Calculate the distance between two atoms


  4%|▍         | 205319/4658147 [00:00<00:04, 1018081.45it/s]

Extracting number of intervene bonds from type


100%|██████████| 2505542/2505542 [00:02<00:00, 1183030.83it/s]


In [23]:
import openbabel

ModuleNotFoundError: No module named 'openbabel'

In [57]:
train.to_csv(os.path.join(dir_out, "_train_0802.csv"), index=False)
print("Saved training dataset to {}".format(os.path.join(dir_out, "_train.csv")))

test.to_csv(os.path.join(dir_out, "_test_0802.csv"), index=False)
print("Saved test dataset to {}".format(os.path.join(dir_out, "_test.csv")))